##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Personalizar lo que sucede en `fit()`

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/keras-team/keras-io/blob/master/tf/customizing_what_happens_in_fit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/customizing_what_happens_in_fit.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/keras-io/tf/customizing_what_happens_in_fit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Descargar cuaderno</a></td>
</table>

## Introducción

Cuando realiza el aprendizaje supervisado, puede usar `fit()` y todo funciona sin problemas.

Cuando necesite escribir su propio ciclo de entrenamiento desde cero, puede usar `GradientTape` y tomar el control de cada pequeño detalle.

Pero, ¿qué sucede si necesita un algoritmo de entrenamiento personalizado, pero aún desea beneficiarse de las funciones convenientes de `fit()` , como devoluciones de llamada, soporte de distribución integrado o fusión de pasos?

Un principio fundamental de Keras es **la divulgación progresiva de la complejidad** . Siempre debe poder ingresar a flujos de trabajo de nivel inferior de manera gradual. No debería caerse de un precipicio si la funcionalidad de alto nivel no coincide exactamente con su caso de uso. Debería poder obtener un mayor control sobre los pequeños detalles mientras conserva una cantidad proporcional de conveniencia de alto nivel.

Cuando necesite personalizar lo que hace `fit()` , debe **anular la función de paso de entrenamiento de la clase `Model`** . Esta es la función a la que llama `fit()` para cada lote de datos. Luego podrá llamar a `fit()` como de costumbre, y ejecutará su propio algoritmo de aprendizaje.

Tenga en cuenta que este patrón no le impide crear modelos con la API funcional. Puede hacer esto ya sea que esté creando modelos `Sequential` , modelos de API funcionales o modelos de subclases.

Veamos cómo funciona.

## Preparar

Requiere TensorFlow 2.2 o posterior.

In [ ]:
import tensorflow as tf
from tensorflow import keras

## Un primer ejemplo sencillo

Comencemos con un ejemplo simple:

- Creamos una nueva clase que subclasifica `keras.Model` .
- We just override the method `train_step(self, data)`.
- Devolvemos los nombres de las métricas de asignación de un diccionario (incluida la pérdida) a su valor actual.

El argumento de entrada `data` es lo que se pasa a encajar como datos de entrenamiento:

- Si pasa matrices Numpy, llamando a `fit(x, y, ...)` , los `data` serán la tupla `(x, y)`
- If you pass a `tf.data.Dataset`, by calling `fit(dataset, ...)`, then `data` will be what gets yielded by `dataset` at each batch.

In the body of the `train_step` method, we implement a regular training update, similar to what you are already familiar with. Importantly, **we compute the loss via `self.compiled_loss`**, which wraps the loss(es) function(s) that were passed to `compile()`.

De manera similar, llamamos `self.compiled_metrics.update_state(y, y_pred)` para actualizar el estado de las métricas que se pasaron en `compile()` , y consultamos los resultados de `self.metrics` al final para recuperar su valor actual.

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


Probemos esto:

In [ ]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

## Yendo a un nivel inferior

Naturalmente, puede omitir pasar una función de pérdida en `compile()` y, en su lugar, hacer todo *manualmente* en `train_step` . Lo mismo ocurre con las métricas. Aquí hay un ejemplo de nivel inferior, que solo usa `compile()` para configurar el optimizador:

In [ ]:
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")
loss_tracker = keras.metrics.Mean(name="loss")


class CustomModel(keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = keras.losses.mean_squared_error(y, y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        loss_tracker.update_state(loss)
        mae_metric.update_state(y, y_pred)
        return {"loss": loss_tracker.result(), "mae": mae_metric.result()}


# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam")

# Just use `fit` as usual -- you can use callbacks, etc.
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

## Compatible con `sample_weight` y `class_weight`

Es posible que haya notado que nuestro primer ejemplo básico no mencionó la ponderación de la muestra. Si desea admitir los argumentos `fit()` `sample_weight` y `class_weight` , simplemente haga lo siguiente:

- Desempaquete `sample_weight` del argumento de `data`
- Páselo a `compiled_loss` & `compiled_metrics` (por supuesto, también puede aplicarlo manualmente si no confía en `compile()` para pérdidas y métricas)
- Eso es. Esa es la lista.

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        if len(data) == 3:
            x, y, sample_weight = data
        else:
            x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.compiled_loss(
                y,
                y_pred,
                sample_weight=sample_weight,
                regularization_losses=self.losses,
            )

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        self.compiled_metrics.update_state(y, y_pred, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# You can now use sample_weight argument
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=3)

## Proporcionar su propio paso de evaluación

¿Qué sucede si desea hacer lo mismo para las llamadas a `model.evaluate()` ? Entonces anularía `test_step` exactamente de la misma manera. Así es como se ve:

In [ ]:
class CustomModel(keras.Model):
    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(loss="mse", metrics=["mae"])

# Evaluate with our custom test_step
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.evaluate(x, y)

## Conclusión: un ejemplo de GAN de extremo a extremo

Veamos un ejemplo de un extremo a otro que aprovecha todo lo que acaba de aprender.

Consideremos:

- Una red generadora destinada a generar imágenes de 28x28x1.
- Una red discriminadora destinada a clasificar imágenes de 28x28x1 en dos clases ("falsas" y "reales").
- Un optimizador para cada uno.
- Una función de pérdida para entrenar al discriminador.


In [ ]:
from tensorflow.keras import layers

# Create the discriminator
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator
latent_dim = 128
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

Aquí hay una clase GAN con todas las funciones, anulando `compile()` para usar su propia firma e implementando todo el algoritmo GAN en 17 líneas en `train_step` :

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        if isinstance(real_images, tuple):
            real_images = real_images[0]
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
        return {"d_loss": d_loss, "g_loss": g_loss}


Probémoslo:

In [ ]:
# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

# To limit execution time, we only train on 100 batches. You can train on
# the entire dataset. You will need about 20 epochs to get nice results.
gan.fit(dataset.take(100), epochs=1)

La idea detrás del aprendizaje profundo es simple, entonces, ¿por qué debería ser dolorosa su implementación?